# YOLO v2训练kmodel

In [1]:
# 把 maix_train 项目的yolo抽离出来
# 基准测试通过
# 来源：https://github.com/sipeed/maix_train

In [2]:
import os, sys
import shutil
import time

from train.detector import Detector
from train.utils.logger import Logger, Fake_Logger
from train.utils import gpu_utils, isascii


In [3]:
from train.detector.instance import config

#config.detector_train_epochs = 40 # 修改训练轮数

## config setting

In [4]:
cur_time = time.strftime("%Y-%m-%d_%H_%M", time.localtime())
print(cur_time)

2021-09-17_15_47


In [5]:
ubuntu_path = f"out/yolo_{cur_time}" #用来mac中生成kmodel
temp_saves_dir = f"../ubuntu/{ubuntu_path}/" #保存文件目录
datasets_dir = "../datasets/ts_xml_format"#数据源
print(temp_saves_dir)

../ubuntu/out/yolo_2021-09-17_15_47/


## 训练函数

In [6]:
def clear_save_path(temp_datasets_dir=""):
    if temp_datasets_dir =="":
        return False
    #clear
    if os.path.exists(temp_datasets_dir):
        shutil.rmtree(temp_datasets_dir)
    os.makedirs(temp_datasets_dir)
    return True

def creat_sample_images(detector,temp_datasets_dir="",sample_image_num = 20):
    dataset_sample_images_path = os.path.join(temp_datasets_dir, "sample_images")
    os.makedirs(dataset_sample_images_path)
    detector.get_sample_images(sample_image_num, dataset_sample_images_path)
    return True

def detector_train(datasets_zip_path="",datasets_dir="",temp_datasets_dir=""):
    
    #clear and creat
    clear_save_path(temp_datasets_dir)
    log = Logger(file_path=f"{temp_datasets_dir}train_log.log")
    
    def __on_train_progress(percent, msg):  # flag: progress
        percent = percent*0.97 + 1
        log.i(f"progress: {percent}%, {msg}")
    
    # 检测 GPU 可用,选择一个可用的 GPU 使用
    try:
        gpu = gpu_utils.select_gpu(memory_require = config.detector_train_gpu_mem_require, tf_gpu_mem_growth=False)
    except Exception:
        gpu = None
    if gpu is None:
        if not config.allow_cpu:
            log.e("no free GPU")
            raise Exception((TrainFailReason.ERROR_NODE_BUSY, "node no enough GPU or GPU memory and not support CPU train"))
        log.i("no GPU, will use [CPU]")
    else:
        log.i("select", gpu)

    # set value
    best_h5_model_path  = os.path.join(temp_datasets_dir, "m_best.h5")
    final_h5_model_path = os.path.join(temp_datasets_dir, "m.h5")
    result_dir = os.path.join(temp_datasets_dir, "result")
    os.makedirs(result_dir)
    result_report_img_path = os.path.join(result_dir, "report.jpg")
    tflite_path = os.path.join(temp_datasets_dir, "m.tflite")
    
    # 启动训练
    try:
        detector = Detector(input_shape=(224, 224, 3),
                            datasets_zip=datasets_zip_path,
                            datasets_dir=datasets_dir,
                            unpack_dir = temp_datasets_dir,
                            logger=log,
                            max_classes_limit = config.detector_train_max_classes_num,
                            one_class_min_images_num=config.detector_train_one_class_min_img_num,
                            one_class_max_images_num=config.detector_train_one_class_max_img_num,
                            allow_reshape = False)
    except Exception as e:
        log.e("train datasets not valid: {}".format(e))
        raise Exception(("TrainFailReason.ERROR_PARAM", "datasets not valid: {}".format(str(e))))
    try:

        f,anchors = detector.train(epochs=config.detector_train_epochs,
                progress_cb= __on_train_progress,
                save_best_weights_path = best_h5_model_path,
                save_final_weights_path = final_h5_model_path,
                jitter=False,
                is_only_detect = False,
                batch_size = config.detector_train_batch_size,
                train_times = 5,
                valid_times = 2,
                learning_rate=config.detector_train_learn_rate,
            )
        print("anchors 2:",anchors)
    except Exception as e:
        log.e("train error: {}".format(e))
        traceback.print_exc()
        raise Exception(("TrainFailReason.ERROR_INTERNAL", "error occurred when train, error: {}".format(str(e)) ))

    ## sample_image
    creat_sample_images(detector,temp_datasets_dir,config.sample_image_num)
    
    ## creat tflite
    detector.save(tflite_path=tflite_path)
    
    # 训练结束, 生成报告
    log.i("train ok, now generate report")
    detector.report(result_report_img_path)
    
    return True , detector,final_h5_model_path,anchors


In [7]:
## 清理隐藏文件
#rm -f /Users/liampro/Downloads/pro/git/ai-test/k210/datasets/ts_xml_format/.DS_Store
# rm -f /Users/liampro/Downloads/pro/git/ai-test/k210/datasets/ts_xml_format/*/.DS_Store
#cmd1 = f"rm -f {datasets_dir}/.DS_Store"
cmd = f"rm -f {datasets_dir}/*/.DS_Store"
#os.popen(cmd1)
f = os.popen(cmd)
print(f)

## 训练

In [8]:
# 配置文件：detector.instance
result,_,model_path,anchors = detector_train(datasets_dir=datasets_dir,temp_datasets_dir=temp_saves_dir)


select gpu fail:NVML Shared Library Not Found


2021-09-17 15:47:03,981 - [INFO]:  no GPU, will use [CPU]
2021-09-17 15:47:07,462 - [INFO]:  input_shape: (224, 224, 3)


-- 0
-- 1


2021-09-17 15:47:08,382 - [INFO]:  load datasets complete, check pass, images num:600, bboxes num:600
2021-09-17 15:47:09,156 - [INFO]:  train, labels:['0', '1']
2021-09-17 15:47:09,157 - [DEBUG]:  train, datasets dir:../datasets/ts_xml_format
2021-09-17 15:47:09,158 - [INFO]:  bboxes num: 600, first bbox: [0.61160714 0.66071429]
2021-09-17 15:47:09,467 - [INFO]:  bbox accuracy(IOU): 85.43%
2021-09-17 15:47:09,468 - [INFO]:  bound boxes: (136.000000,131.00),(105.000000,105.00),(78.000000,77.00),(54.000000,57.00),(165.000000,165.00)
2021-09-17 15:47:09,469 - [INFO]:  anchors: [4.25, 4.09375, 3.28125, 3.28125, 2.4375, 2.40625, 1.6875, 1.78125, 5.15625, 5.15625]
2021-09-17 15:47:09,469 - [INFO]:  w/h ratios: [0.95, 1.0, 1.0, 1.01, 1.04]
2021-09-17 15:47:09.489779: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To ena

self.anchors: [4.25, 4.09375, 3.28125, 3.28125, 2.4375, 2.40625, 1.6875, 1.78125, 5.15625, 5.15625]
load local weight file: /Users/liampro/Downloads/pro/git/ai-test/k210/k210_train/train/detector/weights/mobilenet_7_5_224_tf_no_top.h5


...

INFO:tensorflow:Tensor's key in saved_model's tensor_map: reshape


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 7, 7, 5, 7), type: DT_FLOAT


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 7, 7, 5, 7), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from /var/folders/_l/dffcyljj723fg4wbqfxfqv640000gn/T/tmpmgehu_5u/variables/variables


INFO:tensorflow:Restoring parameters from /var/folders/_l/dffcyljj723fg4wbqfxfqv640000gn/T/tmpmgehu_5u/variables/variables
2021-09-17 16:49:13.645917: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-09-17 16:49:13.645977: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-17 16:49:13.961988: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 1411 nodes (1130), 3074 edges (2522), time = 64.044ms.
  function_optimizer: function_optimizer did nothing. time = 2.394ms.

2021-09-17 16:49:14.657651: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-09-17 16:49:14.657721: I tensorflow/core/g

In [9]:
print(model_path)
print(anchors)

../ubuntu/out/yolo_2021-09-17_15_47/m.h5
[4.25, 4.09375, 3.28125, 3.28125, 2.4375, 2.40625, 1.6875, 1.78125, 5.15625, 5.15625]


In [10]:
print(os.path.abspath('.'))

/Users/liampro/Downloads/pro/git/ai-test/k210/k210_train


## 生成kmodel

In [11]:
#直接生成kmodel:Mac开发环境要弄个docker跑linux命令，
# win,linux可以直接跑官方命令
# tool:https://github.com/kendryte/nncase/releases/tag/v0.1.0-rc5

exe_path = os.path.join(os.path.abspath('..'), "ubuntu")

docker_cmd=f"docker run -it -v {exe_path}:/train ubuntu /train/tokmodel.sh {ubuntu_path}"
print(docker_cmd)

# 在shell 运行 docker命令

docker run -it -v /Users/liampro/Downloads/pro/git/ai-test/k210/ubuntu:/train ubuntu /train/tokmodel.sh out/yolo_2021-09-17_15_47


In [12]:
# 或者运行下面的shell
cmd_sh = f"{os.path.abspath('.')}/tools/kmodel/ncc-mac.sh {exe_path} {ubuntu_path}"
print(cmd_sh)
# f=os.popen(cmd)
# print(f.read())

/Users/liampro/Downloads/pro/git/ai-test/k210/k210_train/tools/kmodel/ncc-mac.sh /Users/liampro/Downloads/pro/git/ai-test/k210/ubuntu out/yolo_2021-09-17_15_47


## 生成k210程序boot.py

In [13]:
#temp_saves_dir = "../ubuntu/out/yolo_1"
target_file = f"{temp_saves_dir}result/boot.py"
copy_file = f"cp -f data/template/yolo_boot.py {target_file}"
print(copy_file)

cp -f data/template/yolo_boot.py ../ubuntu/out/yolo_2021-09-17_15_47/result/boot.py


In [14]:
f=os.popen(copy_file)
print(f.read()) 

In [15]:
# 更新 labels ,anchors
result_boot_py_path = target_file

img_size = [224,224]
labels = ["c0","c1"]
replace = 'labels = ["{}"]'.format('", "'.join(labels))

with open(result_boot_py_path) as f:
            boot_py = f.read()
with open(result_boot_py_path, "w") as f:
            target = 'labels = ["0", "1"] # labels'
            boot_py = boot_py.replace(target, replace)
            target = 'anchors = [3.28125, 3.28125] # anchors'
            replace = 'anchors = [{}]'.format(', '.join(str(i) for i in anchors))
            boot_py = boot_py.replace(target, replace)
            target = 'img_size = [224,224] # img_size'
            replace = 'img_size = [{},{}]'.format(img_size[0], img_size[1])
            boot_py = boot_py.replace(target, replace)
            f.write(boot_py)

## 最后生成的文件tree

## 到k210硬件运行